<a href="https://colab.research.google.com/github/Maharaniputrie/FinPro_Karir.mu/blob/main/Fix_Praproses_Alpha_FreeA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

inarhehehe_sensor_3863_path = kagglehub.dataset_download('inarhehehe/sensor-3863')
inarhehehe_sensor_3865_path = kagglehub.dataset_download('inarhehehe/sensor-3865')
inarhehehe_sensor_3867_path = kagglehub.dataset_download('inarhehehe/sensor-3867')

print('Data source import complete.')


# Praproses Data
1. Formatting Data
2. Duplikat
3. Null
4. merging interpolated + value_tag
5. Outlier
6. cek time_stamp yang hilang

In [ ]:
import pandas as pd
!pip install nolds
import nolds
import matplotlib.pyplot as plt

In [ ]:
sensor3865= pd.read_csv('/kaggle/input/sensor-3865/interpolated_3865.csv', delimiter=',')
sensor3865_1= pd.read_csv('/kaggle/input/sensor-3865/dl_value_tag_3865.csv', delimiter=',')
sensor3863= pd.read_csv('/kaggle/input/sensor-3863/interpolated_3863.csv', delimiter=',')
sensor3863_1= pd.read_csv('/kaggle/input/sensor-3863/dl_value_tag_3863.csv', delimiter=',')
sensor3867= pd.read_csv('/kaggle/input/sensor-3867/interpolated_3867.csv', delimiter=',')
sensor3867_1= pd.read_csv('/kaggle/input/sensor-3867/dl_value_tag_3867.csv', delimiter=',')

In [ ]:
sensor3865 = sensor3865[['tag_id', 'time_stamp', 'value']] ##interpolated
sensor3865_1 = sensor3865_1[['tag_id', 'time_stamp', 'value']]
sensor3863 = sensor3863[['tag_id', 'time_stamp', 'value']] ##interpolated
sensor3863_1 = sensor3863_1[['tag_id', 'time_stamp', 'value']]
sensor3867 = sensor3867[['tag_id', 'time_stamp', 'value']] ##interpolated
sensor3867_1 = sensor3867_1[['tag_id', 'time_stamp', 'value']]

In [ ]:
# Daftar DataFrame sensor yang ingin dicek
sensor_list = [sensor3865, sensor3865_1, sensor3863, sensor3863_1, sensor3867, sensor3867_1]
sensor_names = ["sensor3865", "sensor3865_1", "sensor3863", "sensor3863_1", "sensor3867", "sensor3867_1"]

# Loop untuk menampilkan info dari setiap sensor
for name, df in zip(sensor_names, sensor_list):
    print(f"===== {name} =====")
    print(df.info())  # Menampilkan informasi DataFrame
    print("\n")  # Menambahkan baris kosong agar lebih rapi


===== sensor3865 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485291 entries, 0 to 485290
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tag_id      485291 non-null  int64  
 1   time_stamp  485291 non-null  object 
 2   value       485291 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 11.1+ MB
None


===== sensor3865_1 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111243 entries, 0 to 111242
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tag_id      111243 non-null  int64  
 1   time_stamp  111243 non-null  object 
 2   value       111243 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.5+ MB
None


===== sensor3863 =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485291 entries, 0 to 485290
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 

# **Sensor 3865**

### Formatting Data

In [ ]:
# Pastikan 'time_stamp' dalam format datetime
sensor3865['time_stamp'] = pd.to_datetime(sensor3865['time_stamp'])
sensor3865 = sensor3865.sort_values(by='time_stamp')
sensor3865.info()

<class 'pandas.core.frame.DataFrame'>
Index: 485291 entries, 0 to 485290
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   tag_id      485291 non-null  int64         
 1   time_stamp  485291 non-null  datetime64[ns]
 2   value       485291 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 14.8 MB


In [ ]:
# Pastikan 'time_stamp' dalam format datetime
sensor3865_1['time_stamp'] = pd.to_datetime(sensor3865_1['time_stamp'])
sensor3865_1 = sensor3865_1.sort_values(by='time_stamp')
sensor3865_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 111243 entries, 110648 to 110643
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   tag_id      111243 non-null  int64         
 1   time_stamp  111243 non-null  datetime64[ns]
 2   value       111243 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.4 MB


### **Duplikat**

In [ ]:
print(f"Total Duplicates: {sensor3865.duplicated().sum()}")
print(f"nilai null: {sensor3865.isnull().sum()}")

Total Duplicates: 82089
nilai null: tag_id        0
time_stamp    0
value         0
dtype: int64


In [ ]:
# Filter hanya yang duplikat
duplikat = sensor3865[sensor3865.duplicated(keep=False)]
print(duplikat)

> Before after drop duplicate

In [ ]:
# Simpan DataFrame sebelum drop duplicate
sensor_before = sensor3865.copy()

# Hapus duplikasi berdasarkan 'time_stamp' (ambil yang pertama)
sensor3865 = sensor3865.drop_duplicates(subset=['time_stamp'], keep='first')

# Simpan DataFrame setelah drop duplicate
sensor_after = sensor3865.copy()

# Buat subplot untuk Before & After Drop Duplicate
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot sebelum drop duplicate
axes[0].plot(sensor_before['time_stamp'], sensor_before['value'], linestyle='-', label='Before Drop Duplicate', color='red')
axes[0].set_title('Plot Data Sensor - Before Drop Duplicate')
axes[0].set_ylabel('Sensor Value')
axes[0].legend()
axes[0].grid()

# Plot sesudah drop duplicate
axes[1].plot(sensor_after['time_stamp'], sensor_after['value'], linestyle='-', label='After Drop Duplicate', color='blue')
axes[1].set_title('Plot Data Sensor - After Drop Duplicate')
axes[1].set_xlabel('Timestamp')
axes[1].set_ylabel('Sensor Value')
axes[1].legend()
axes[1].grid()

# Rotasi label x agar lebih terbaca
plt.xticks(rotation=45)

# Tampilkan plot
plt.tight_layout()
plt.show()

--------------------------------------------------------------

In [ ]:
print(f"Total Duplicates: {sensor3865_1.duplicated().sum()}")
print(f"nilai null: {sensor3865_1.isnull().sum()}")

In [ ]:
# Filter hanya yang duplikat
duplikat = sensor3865_1[sensor3865_1.duplicated(keep=False)]
print(duplikat)

In [ ]:
# Simpan DataFrame sebelum drop duplicate
sensor_before = sensor3865_1.copy()

# Hapus duplikasi berdasarkan 'time_stamp' (ambil yang pertama)
sensor3865_1 = sensor3865_1.drop_duplicates(subset=['time_stamp'], keep='first')

# Simpan DataFrame setelah drop duplicate
sensor_after1 = sensor3865_1.copy()

# Buat subplot untuk Before & After Drop Duplicate
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot sebelum drop duplicate
axes[0].plot(sensor_before['time_stamp'], sensor_before['value'], linestyle='-', label='Before Drop Duplicate', color='red')
axes[0].set_title('Plot Data Sensor - Before Drop Duplicate')
axes[0].set_ylabel('Sensor Value')
axes[0].legend()
axes[0].grid()

# Plot sesudah drop duplicate
axes[1].plot(sensor_after1['time_stamp'], sensor_after1['value'], linestyle='-', label='After Drop Duplicate', color='blue')
axes[1].set_title('Plot Data Sensor - After Drop Duplicate')
axes[1].set_xlabel('Timestamp')
axes[1].set_ylabel('Sensor Value')
axes[1].legend()
axes[1].grid()

# Rotasi label x agar lebih terbaca
plt.xticks(rotation=45)

# Tampilkan plot
plt.tight_layout()
plt.show()

### Merging

In [ ]:
full3865 = pd.concat([sensor_after,sensor_after1], ignore_index=True)
full3865.shape

In [ ]:
# Plot data sensor
plt.figure(figsize=(10, 5))
plt.plot(full3865['time_stamp'], full3865['value'], linestyle='-', label='Sensor Value')

# Format plot
plt.xlabel('Timestamp')
plt.ylabel('Sensor Value')
plt.title('Plot Data Sensor 3865')
plt.legend()
plt.xticks(rotation=45)  # Rotasi label waktu agar lebih terbaca
plt.grid()

# Tampilkan plot
plt.show()

In [ ]:
print(f"Total Duplicates: {full3865.duplicated().sum()}")
print(f"nilai null: {full3865.isnull().sum()}")

### Outlier

In [ ]:
# Filter data antara 2024-01-01 hingga 2024-12-31
full3865 = full3865[
    (full3865['time_stamp'] >= '2024-01-01') &
    (full3865['time_stamp'] <= '2024-12-31')
].reset_index(drop=True)

# Filter data dengan nilai 'value' di bawah atau sama dengan 120
full3865_cleaned = full3865[full3865['value'] <= 30].reset_index(drop=True)
full3865_cleaned.shape

# Urutkan berdasarkan timestamp
full3865_cleaned = full3865_cleaned.sort_values(by='time_stamp')

# Plot data sensor
plt.figure(figsize=(10, 5))
plt.plot(full3865_cleaned['time_stamp'], full3865_cleaned['value'], linestyle='-', label='Sensor Value')

# Format plot
plt.xlabel('Timestamp')
plt.ylabel('Sensor Value')
plt.title('Plot Data Sensor 3863')
plt.legend()
plt.xticks(rotation=45)  # Rotasi label waktu agar lebih terbaca
plt.grid()

# Tampilkan plot
plt.show()

### Cek time_stamp hilang

In [ ]:
# Buat kolom 'date' yang hanya berisi tanggal (tanpa jam & menit)
full3865_cleaned['date'] = full3865_cleaned['time_stamp'].dt.date

# Buat rentang tanggal lengkap dari min ke max date untuk mengecek missing dates
full_date_range = pd.date_range(start=full3865_cleaned['date'].min(),
                                end=full3865_cleaned['date'].max(), freq='D').date  # Gunakan hanya hari

# Cari tanggal yang hilang dengan membandingkan dengan rentang lengkap
missing_dates = pd.DataFrame({'date': full_date_range[~pd.Series(full_date_range).isin(full3865_cleaned['date'])]})
# Menampilkan semua daftar tanggal yang hilang tanpa truncation
pd.set_option('display.max_rows', None)  # Mengatur agar semua baris ditampilkan

# Menampilkan daftar tanggal yang hilang
print("Tanggal yang tidak ada dalam dataset (dd-mm-yy):")
print(missing_dates)

# Sensor 3867

### Formatting Data

In [ ]:
# Pastikan 'time_stamp' dalam format datetime
sensor3867['time_stamp'] = pd.to_datetime(sensor3867['time_stamp'])
sensor3867 = sensor3867.sort_values(by='time_stamp')
sensor3867.info()

In [ ]:
# Pastikan 'time_stamp' dalam format datetime
sensor3867_1['time_stamp'] = pd.to_datetime(sensor3867_1['time_stamp'])
sensor3867_1 = sensor3867_1.sort_values(by='time_stamp')
sensor3867_1.info()

### Duplikat

In [ ]:
print(f"Total Duplicates: {sensor3867.duplicated().sum()}")
print(f"nilai null: {sensor3867.isnull().sum()}")

In [ ]:
# Filter hanya yang duplikat
duplikat = sensor3867[sensor3867.duplicated(keep=False)]
print(duplikat)

In [ ]:
# Simpan DataFrame sebelum drop duplicate
sensor_before = sensor3867.copy()

# Hapus duplikasi berdasarkan 'time_stamp' (ambil yang pertama)
sensor3867 = sensor3867.drop_duplicates(subset=['time_stamp'], keep='first')

# Simpan DataFrame setelah drop duplicate
sensor_after2 = sensor3867.copy()

# Buat subplot untuk Before & After Drop Duplicate
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot sebelum drop duplicate
axes[0].plot(sensor_before['time_stamp'], sensor_before['value'], linestyle='-', label='Before Drop Duplicate', color='red')
axes[0].set_title('Plot Data Sensor - Before Drop Duplicate')
axes[0].set_ylabel('Sensor Value')
axes[0].legend()
axes[0].grid()

# Plot sesudah drop duplicate
axes[1].plot(sensor_after2['time_stamp'], sensor_after2['value'], linestyle='-', label='After Drop Duplicate', color='blue')
axes[1].set_title('Plot Data Sensor - After Drop Duplicate')
axes[1].set_xlabel('Timestamp')
axes[1].set_ylabel('Sensor Value')
axes[1].legend()
axes[1].grid()

# Rotasi label x agar lebih terbaca
plt.xticks(rotation=45)

# Tampilkan plot
plt.tight_layout()
plt.show()

--------------------------------------------------------------

In [ ]:
print(f"Total Duplicates: {sensor3867_1.duplicated().sum()}")
print(f"nilai null: {sensor3867_1.isnull().sum()}")

In [ ]:
# Filter hanya yang duplikat
duplikat = sensor3867_1[sensor3867_1.duplicated(keep=False)]
print(duplikat)

In [ ]:
# Simpan DataFrame sebelum drop duplicate
sensor_before = sensor3867.copy()

# Hapus duplikasi berdasarkan 'time_stamp' (ambil yang pertama)
sensor3867 = sensor3867.drop_duplicates(subset=['time_stamp'], keep='first')

# Simpan DataFrame setelah drop duplicate
sensor_after3 = sensor3867.copy()

# Buat subplot untuk Before & After Drop Duplicate
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot sebelum drop duplicate
axes[0].plot(sensor_before['time_stamp'], sensor_before['value'], linestyle='-', label='Before Drop Duplicate', color='red')
axes[0].set_title('Plot Data Sensor - Before Drop Duplicate')
axes[0].set_ylabel('Sensor Value')
axes[0].legend()
axes[0].grid()

# Plot sesudah drop duplicate
axes[1].plot(sensor_after3['time_stamp'], sensor_after3['value'], linestyle='-', label='After Drop Duplicate', color='blue')
axes[1].set_title('Plot Data Sensor - After Drop Duplicate')
axes[1].set_xlabel('Timestamp')
axes[1].set_ylabel('Sensor Value')
axes[1].legend()
axes[1].grid()

# Rotasi label x agar lebih terbaca
plt.xticks(rotation=45)

# Tampilkan plot
plt.tight_layout()
plt.show()

### Merging

In [ ]:
full3867 = pd.concat([sensor_after2,sensor_after3], ignore_index=True)
full3867.shape

In [ ]:
# Plot data sensor
plt.figure(figsize=(10, 5))
plt.plot(full3867['time_stamp'], full3867['value'], linestyle='-', label='Sensor Value')

# Format plot
plt.xlabel('Timestamp')
plt.ylabel('Sensor Value')
plt.title('Plot Data Sensor 3865')
plt.legend()
plt.xticks(rotation=45)  # Rotasi label waktu agar lebih terbaca
plt.grid()

# Tampilkan plot
plt.show()

In [ ]:
print(f"Total Duplicates: {full3867.duplicated().sum()}")
print(f"nilai null: {full3867.isnull().sum()}")

### Outlier

In [ ]:
# Filter data antara 2024-01-01 hingga 2024-12-31
full3867 = full3867[
    (full3867['time_stamp'] >= '2024-01-01') &
    (full3867['time_stamp'] <= '2024-12-31')
].reset_index(drop=True)

# Filter data dengan nilai 'value' di bawah atau sama dengan 120
full3867_cleaned = full3867[full3867['value'] <= 150].reset_index(drop=True)
full3867_cleaned.shape

# Urutkan berdasarkan timestamp
full3867_cleaned = full3867_cleaned.sort_values(by='time_stamp')

# Plot data sensor
plt.figure(figsize=(10, 5))
plt.plot(full3867_cleaned['time_stamp'], full3867_cleaned['value'], linestyle='-', label='Sensor Value')

# Format plot
plt.xlabel('Timestamp')
plt.ylabel('Sensor Value')
plt.title('Plot Data Sensor 3863')
plt.legend()
plt.xticks(rotation=45)  # Rotasi label waktu agar lebih terbaca
plt.grid()

# Tampilkan plot
plt.show()

### Cek time_stamp hilang

In [ ]:
# Buat kolom 'date' yang hanya berisi tanggal (tanpa jam & menit)
full3867_cleaned['date'] = full3867_cleaned['time_stamp'].dt.date

# Buat rentang tanggal lengkap dari min ke max date untuk mengecek missing dates
full_date_range = pd.date_range(start=full3867_cleaned['date'].min(),
                                end=full3867_cleaned['date'].max(), freq='D').date  # Gunakan hanya hari

# Cari tanggal yang hilang dengan membandingkan dengan rentang lengkap
missing_dates = pd.DataFrame({'date': full_date_range[~pd.Series(full_date_range).isin(full3867_cleaned['date'])]})
# Menampilkan semua daftar tanggal yang hilang tanpa truncation
pd.set_option('display.max_rows', None)  # Mengatur agar semua baris ditampilkan

# Menampilkan daftar tanggal yang hilang
print("Tanggal yang tidak ada dalam dataset (dd-mm-yy):")
print(missing_dates)